In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
pd.options.mode.chained_assignment = None

# 1. Dataset

In [ ]:
# Cargamos el dataset desde el link
paises = pd.read_table('https://raw.githubusercontent.com/jennybc/gapminder/master/data-raw/08_gap-every-five-years.tsv')

In [ ]:
# Vemos 10 registros al azar
paises.sample(10)

In [ ]:
print(f"El primer año en el dataset es {paises.year.min()} y el último año es {paises.year.max()}")

El dataset contiene información de expectativa de vida, población y PBI per capita de países desde 1952 hasta 2007 en intervalos de 5 años. Vamos a trabajar con la información más reciente.

In [ ]:
# Nos quedamos con la información para el año 2007
paises_2007 = paises.query("year==2007")

Vamos a utilizar el método `pandas.describe()` para conocer las variables numéricas del dataset 

In [ ]:
paises_2007.describe()

Vemos que hay diferencias importantes entre los valores minimos y maximos de cada variable.

También ahora podemos observar cuántos paises hay por continente. Este análisis lo podemos realizar con la función `value_counts()` o mediante `groupby(columna).size()`

In [ ]:
# Cantidad de países por continente: opción 1
paises_2007.continent.value_counts()

In [ ]:
# Cantidad de países por continente: opción 2
paises_2007.groupby('continent').size()

# 2. Discretización

Ahora que ya conocemos el dataset, nos interesa poder discretizar las variables de expectativa de vida, población y PBI per capita para agrupar a los países

## 2.1 Población

En el caso de la población nos interesa agruparlos por un criterio propio en las siguientes categorías:

* muy pequeño: hasta 1 millon de personas
* pequeño: desde 1 millon hasta 7.5 millones de personas
* mediano: desde 7.5 millones hasta 25 millones de personas
* grande: desde 25 millones hasta 100 millones de personas
* enorme: desde 100 millones

Revisemos la información del comando `pandas.cut()`

In [ ]:
# Abrimos la ayuda sobre la función
?pd.cut()

pandas.cut(x, bins, right=True, labels=None, ordered=True)

En este caso **x** es la columna de población y en **bins** le pasamos los límites que hemos definido

In [ ]:
# Definimos los limites de los intervalos
bins_poblacion = [0, 1_000_000, 7_500_000, 25_000_000, 100_000_000, np.inf]

* Usamos el guion bajo para separar los números y que sean más faciles de leer
* Usamos `np.inf` para denotar que no definimos el ĺímite superior 

In [ ]:
# Discretizamos la variable de población
pd.cut(x=paises_2007['pop'],bins=bins_poblacion)

Observamos que la función nos devuelve la variable transformada y las categorías ordenadas. Para mejorar la interpretación podemos agregar los nombres en el argumento **labels**

In [ ]:
# Lista de categorías
etiquetas_poblacion =  ['muy pequeño','pequeño', 'mediano', 'grande','enorme']
# Discretizamos la variable de población
pd.cut(x=paises_2007['pop'],bins=bins_poblacion, labels=etiquetas_poblacion)

Ahora que ya tenemos nuestras categorías, podemos crear una nueva variable con la población discretizada

In [ ]:
# Discretizamos la variable de población
paises_2007['cat_poblacion'] = pd.cut(x=paises_2007.loc[:,'pop'],bins=bins_poblacion, labels=etiquetas_poblacion)

In [ ]:
# Observamos la nueva columna
paises_2007.sample(10)

## 2.2 Expectativa de vida

En el caso de la expectativa de vida nos interesa agruparlos en 5 categorías: muy baja, baja, media, alta y muy alta. Pero en este caso vamos a querer comparar los bins que surjan del método por igual frecuencia o por igual ancho.

### 2.2.1 Intervalos de igual ancho

Para crear intervalos de igual ancho tenemos que utilizar la función `pandas.cut()`. En este caso le pasamos la cantidad de bins que queremos, 5, al argumento **bins**

In [ ]:
# Discretizamos la variable de expectativa de vida
pd.cut(x=paises_2007.lifeExp, bins=5)

In [ ]:
# Guardamos los bins para comparar 
_ , bins_igual_ancho = pd.cut(x=paises_2007.lifeExp, bins=5, retbins=True)

### 2.2.2 Intervalos de igual frecuencia

Para crear intervalos de igual ancho tenemos que utilizar la función `pandas.qcut()`. En este caso le pasamos la cantidad de bins que queremos, 5, al argumento **q**

In [ ]:
# Abrimos la ayuda sobre la función
? pd.qcut()

In [ ]:
# Discretizamos la variable de expectativa de vida
pd.qcut(x=paises_2007.lifeExp, q=5)

In [ ]:
# Guardamos los bins para comparar 
_ , bins_igual_frecuencia = pd.qcut(x=paises_2007.lifeExp, q=5, retbins=True)

### 2.2.3 Comparación de los dos métodos

In [ ]:
# Bins de igual ancho
bins_igual_ancho

In [ ]:
# Bins de igual frecuencia
bins_igual_frecuencia

Los intervalos de igual ancho se encuentran definidos por aproximadamente 8.5 años pero tienen una cantidad variable de paises. 

Los intervalos de igual frecuencia (en este caso quintiles) tienen una longitud variable, pero en cada uno de ellos se encuentra aproximadamente el 20% de los países

In [ ]:
# Categorías expectativa de vida
etiquetas_exp_vida = ['muy_baja', 'baja', 'media', 'alta', 'muy_alta']
# Vamos a crear una variable nueva para cada tipo de discretizacion
paises_2007['cat_exp_vida_ancho'] = pd.cut(x=paises_2007['lifeExp'], bins=5, labels=etiquetas_exp_vida)
paises_2007['cat_exp_vida_frec'] = pd.qcut(x=paises_2007['lifeExp'], q=5, labels=etiquetas_exp_vida)

In [ ]:
# Observemos nuestros datos
paises_2007.sample(10)

## 2.3 PBI per capita

En el caso de esta variable nos interesa probar cual es el resultado de utilizar alguno de los métodos de la función `numpy.histogram()`

In [ ]:
# Abrimos la ayuda sobre la función
?np.histogram()

La función `numpy.histogram()` nos devuelve dos elementos:

* La lista con la cantidad de elementos en cada bin
* Los límites de los intervalos

In [ ]:
# Calculamos los bins con el método de Sturges
np.histogram(paises_2007.gdpPercap, bins='sturges')

In [ ]:
# Guardamos la información en dos variables
elementos, limites_pbi = np.histogram(paises_2007.gdpPercap, bins='sturges')

In [ ]:
print(f"El histograma cuenta con {len(elementos)} intervalos")

In [ ]:
# Para crear la variable discretizada le pasamos la lista de limites a pd.cut()
pd.cut(x=paises_2007.gdpPercap, bins=limites_pbi)

In [ ]:
# Para facilitar la interpretación generamos etiquetas
pd.cut(x=paises_2007.gdpPercap, bins=limites_pbi, labels=range(1,10))

In [ ]:
# Creamos la nueva variable discretizada
paises_2007['cat_pbi_per_cap'] = pd.cut(x=paises_2007.gdpPercap, bins=limites_pbi, labels=range(1,10))

In [ ]:
# Veamos como luce nuestro dataset final
paises_2007.sample(10)

# 3 Análisis de las variables discretizadas

Veamos cuantos países están en las nuevas categorías y algunos gráficos sencillos

In [ ]:
#¿Cuántos países hay en cada categoría de población?
paises_2007.groupby('cat_poblacion').size()

In [ ]:
paises_2007.groupby('cat_poblacion').size().plot.bar()

In [ ]:
#¿Cuántos países hay en cada categoría de población en cada continente?
paises_2007.groupby(['continent','cat_poblacion']).size()

In [ ]:
# Graficamos
cantidad_pob_cont = paises_2007.groupby(['continent','cat_poblacion']).agg(cantidad=('country', 'count')).reset_index()
sns.barplot(x='continent', y='cantidad', hue='cat_poblacion', data=cantidad_pob_cont)

In [ ]:
# ¿Cuántos países hay para las categorias de esperanza de vida de igual ancho?
paises_2007.groupby('cat_exp_vida_ancho').size()

In [ ]:
paises_2007.groupby('cat_exp_vida_ancho').size().plot.bar(color='green')

In [ ]:
# ¿Cuántos países hay para las categorias de esperanza de vida de igual frecuencia?
paises_2007.groupby('cat_exp_vida_frec').size()

In [ ]:
paises_2007.groupby('cat_exp_vida_ancho').size().plot.bar(color='firebrick')

In [ ]:
#¿Cuántos países hay para las categorías de pbi per capita y esperanza de vida?
paises_2007.groupby(['cat_exp_vida_frec', 'cat_pbi_per_cap']).size()

In [ ]:
#Veamos los paises que tiene expectativa muy alto con bajos ingresos

In [ ]:
paises_2007.query('cat_exp_vida_ancho == "muy_alta" and cat_pbi_per_cap <= 3')